In [1]:
import pdfplumber
from fastcore.all import *
from configparser import ConfigParser

In [2]:
config = ConfigParser()
config.read('settings.ini')
cfg = config['DEFAULT']

In [6]:
def download_pdf(url, dest):
    if not Path(dest).exists(): urlsave(url, dest)
    return dest

In [7]:
path = download_pdf(cfg['url'], cfg['dest'])

In [50]:
def get_pdf_text(path):
    with pdfplumber.open(path) as pdf: res = L(pdf.pages).map(lambda x: x.extract_text())
    return res

In [51]:
pdf_text = get_pdf_text(path)

In [78]:
def remove_header(s): return remove_prefix(s, 'La Constitución \n')

In [301]:
pat_pagenum = re.compile(r'\n\s+\d+\s*$')
def remove_pagenum(s): return pat_pagenum.sub('', s)

In [303]:
def process_page(s):
    pipe = [remove_header, remove_pagenum]
    for f in pipe: s = f(s)
    return s

In [304]:
proc_pdf_text = pdf_text.map(process_page)

In [491]:
pat_footnote = re.compile(r'\n\d+\s(?:.|\s)*')
pat_footnote_n = re.compile(r'(\d+)\s.*')

def split_footnote(s):
    mat = pat_footnote_n.match(s)
    return (int(mat.group(1)),s[mat.end(1):].strip())

def extract_footnotes(s):
    footnote = pat_footnote.search(s)
    footnotes = []
    
    if footnote is not None:
        footnotes.append(footnote.group().strip())
        mat = pat_footnote_n.match(footnotes[0])
        n = int(mat.group(1))
        
        while True:
            pat = f'\n({int(mat.group(1))+1}) '
            mat = re.search(pat, footnotes[-1])
            if mat is None: break
            i = mat.start(1)
            footnotes.append(footnotes[-1][i:])
            footnotes[-2] = footnotes[-2][:i]
            
        footnotes = L(footnotes).map(lambda x: x.strip().replace('\n', '')).map(split_footnote)
    
    new_s = pat_footnote.sub('',s)
    return new_s,dict(footnotes)

In [ ]:
# TODO add ref link
# 28 -> #28#

In [500]:
L(extract_footnotes(proc_pdf_text[44])).map(print)

1. Ser peruano de nacimiento; 
 
2. Ser ciudadano en ejercicio; 
 
3. Ser mayor de cuarenta y cinco años; 
 
4. Haber sido magistrado de la Corte Superior o Fiscal Superior durante diez años, o haber 
ejercido la abogacía o la cátedra  universitaria en materia jurídica durante quince años. 
 
Artículo 148.- Acción contencioso-administrativa 
Las  resoluciones  administrativas  que causan estado  son  susceptibles de impugnación 
mediante la acción contencioso-administrativa. 
 
Artículo 149.- Ejercicio de la función jurisdiccional por las comunidades campesinas y 
nativas 
Las autoridades de las Comunidades Campesinas y Nativas, con el apoyo de las Rondas 
Campesinas, pueden ejercer las funciones jurisdiccionales dentro de su ámbito territorial 
de conformidad con el derecho consuetudinario, siempre que no violen los derechos 
fundamentales  de  la  persona.  La  ley  establece  las  formas  de  coordinación  de  dicha 
jurisdicción especial con los Juzgados de Paz y con las demás inst

(#2) [None,None]

In [487]:
extract_footnotes(proc_pdf_text[3])[1]

[]

In [373]:
t = pat_footnote_n.match(extract_footnotes(proc_pdf_text[-1])[1])
t

<re.Match object; span=(0, 126), match='46 Disposición Transitoria Especial agregada por >
-1


<re.Match object; span=(0, 126), match='46 Disposición Transitoria Especial agregada por >

In [394]:
ww = extract_footnotes(proc_pdf_text[-1])[1]
ww

<re.Match object; span=(0, 126), match='46 Disposición Transitoria Especial agregada por >
-1


'46 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano.  \n47 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano. \n48 Disposición incorporada por el artículo único de la Ley N.º 29402, publicada el 8 septiembre 2009. La reforma constitucional entró en \nvigencia para el proceso electoral del año 2011.'

In [399]:
ww

'46 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano.  \n47 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano. \n48 Disposición incorporada por el artículo único de la Ley N.º 29402, publicada el 8 septiembre 2009. La reforma constitucional entró en \nvigencia para el proceso electoral del año 2011.'

In [418]:
t.start(1)

0

In [417]:
t.end(1)

2

In [384]:
t.endpos

497

In [381]:
type(t.group(1))

str

In [370]:
type(t)

NoneType

In [343]:
pat_footnote = re.compile(r'\n\d+\s(?:.|\s)*')
qq = pat_footnote.search(proc_pdf_text[-1])
qq.group()

'\n46 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano.  \n47 Disposición Transitoria Especial agregada por el artículo 2 de la Ley N.º 27365, publicada el 5 de noviembre de 2000 en el \ndiario oficial El Peruano. \n48 Disposición incorporada por el artículo único de la Ley N.º 29402, publicada el 8 septiembre 2009. La reforma constitucional entró en \nvigencia para el proceso electoral del año 2011. '

In [296]:
print(proc_pdf_text[-1])

Las  disposiciones  contenidas  en  la  presente  Constitución,  referidas  a  número  de 
congresistas, duración del mandato legislativo, y Comisión Permanente, no se aplican para 
el Congreso Constituyente Democrático. 
 
Decimosexta.- Sustitución de la Constitución de 1979 
Promulgada la presente Constitución, sustituye a la del año 1979. 
 
 
DISPOSICIONES TRANSITORIAS ESPECIALES  
 
Primera.- El Presidente y los Vicepresidentes de la República elegidos en las Elecciones 
Generales de 2000, concluirán su mandato el 28 de julio de 2001. Los congresistas elegidos 
en el mismo proceso electoral culminarán su representación el 26 de julio de 2001. No son 
de aplicación para ellos, por excepción, los plazos establecidos en los Artículos 90 y 112 de 
la Constitución Política.46  
 
Segunda.- Para efectos del proceso electoral que se realice en el 2001, el plazo previsto en 
el primer párrafo del Artículo 91 de la Constitución será de cuatro meses.47 
 
Tercera.- El Jurado Nacional de Ele

In [266]:
pat_footnote = re.compile(r'\n\d+\s(.|\n)*$')
q = pat_footnote.search(process_page(t))
q

<re.Match object; span=(1541, 1771), match='\n1 De conformidad con el artículo 1 de la Ley N.>

In [283]:
pat_footnote.sub()

'\n1 De conformidad con el artículo 1 de la Ley N.º 27600, publicada el 16 diciembre 2001 en el diario oficial El Peruano, se suprime la \nfirma de Alberto Fujimori Fujimori, del texto de la Constitución Política del Estado de 1993. '

In [284]:
q.group()

'\n1 De conformidad con el artículo 1 de la Ley N.º 27600, publicada el 16 diciembre 2001 en el diario oficial El Peruano, se suprime la \nfirma de Alberto Fujimori Fujimori, del texto de la Constitución Política del Estado de 1993. '

In [252]:
process_page(t.replace('º',''))[-300:]

'edio de comunicación \n                                                 \n1 De conformidad con el artículo 1 de la Ley N. 27600, publicada el 16 diciembre 2001 en el diario oficial El Peruano, se suprime la \nfirma de Alberto Fujimori Fujimori, del texto de la Constitución Política del Estado de 1993. '

In [166]:
t

'La Constitución \nPREAMBULO \n \n  El  Congreso  Constituyente  Democrático,  invocando  a  Dios  Todopoderoso, \nobedeciendo  el  mandato  del  pueblo  peruano  y  recordando  el  sacrificio  de  todas  las \ngeneraciones  que  nos  han  precedido  en  nuestra  Patria,  ha  resuelto  dar  la  siguiente \nConstitución: \n \nCONSTITUCION POLITICA DEL PERU DE 1993 1 \n \n \nTITULO I \n \nDE LA PERSONA Y DE LA SOCIEDAD \n \n \nCAPITULO I \n \nDERECHOS FUNDAMENTALES DE LA PERSONA \n \nArtículo  1.- Defensa de la persona humana \nLa defensa de la persona humana y el respeto de su dignidad son el fin supremo de la \nsociedad y del Estado. \n \nArtículo  2.- Derechos fundamentales de la persona \nToda persona tiene derecho: \n \n1. A la vida, a su identidad, a su integridad moral, psíquica y  física y a su libre desarrollo \ny bienestar. El concebido es sujeto de derecho en todo cuanto le favorece. \n \n2. A la igualdad ante la ley. Nadie debe ser discriminado por motivo de origen, raza, sex

In [156]:
print(''.join(pdf_text.map(lambda x: process_page(x)+'#')))

PREAMBULO 
 
  El  Congreso  Constituyente  Democrático,  invocando  a  Dios  Todopoderoso, 
obedeciendo  el  mandato  del  pueblo  peruano  y  recordando  el  sacrificio  de  todas  las 
generaciones  que  nos  han  precedido  en  nuestra  Patria,  ha  resuelto  dar  la  siguiente 
Constitución: 
 
CONSTITUCION POLITICA DEL PERU DE 1993 1 
 
 
TITULO I 
 
DE LA PERSONA Y DE LA SOCIEDAD 
 
 
CAPITULO I 
 
DERECHOS FUNDAMENTALES DE LA PERSONA 
 
Artículo  1.- Defensa de la persona humana 
La defensa de la persona humana y el respeto de su dignidad son el fin supremo de la 
sociedad y del Estado. 
 
Artículo  2.- Derechos fundamentales de la persona 
Toda persona tiene derecho: 
 
1. A la vida, a su identidad, a su integridad moral, psíquica y  física y a su libre desarrollo 
y bienestar. El concebido es sujeto de derecho en todo cuanto le favorece. 
 
2. A la igualdad ante la ley. Nadie debe ser discriminado por motivo de origen, raza, sexo, 
idioma, religión, opinión, condición económi

In [148]:
pdf_text.map(lambda x: x[-10:])

(#67) ['993. \n  1 ',' las \n  2 ','d. \n \n  3 ','ades \n  4 ','ano. \n  5 ','ano, \n  6 ','ano. \n  7 ','era. \n  8 ','ico, \n  9 ','io. \n  10 '...]

In [72]:
t = pdf_text[0]
print(t)

La Constitución 
PREAMBULO 
 
  El  Congreso  Constituyente  Democrático,  invocando  a  Dios  Todopoderoso, 
obedeciendo  el  mandato  del  pueblo  peruano  y  recordando  el  sacrificio  de  todas  las 
generaciones  que  nos  han  precedido  en  nuestra  Patria,  ha  resuelto  dar  la  siguiente 
Constitución: 
 
CONSTITUCION POLITICA DEL PERU DE 1993 1 
 
 
TITULO I 
 
DE LA PERSONA Y DE LA SOCIEDAD 
 
 
CAPITULO I 
 
DERECHOS FUNDAMENTALES DE LA PERSONA 
 
Artículo  1.- Defensa de la persona humana 
La defensa de la persona humana y el respeto de su dignidad son el fin supremo de la 
sociedad y del Estado. 
 
Artículo  2.- Derechos fundamentales de la persona 
Toda persona tiene derecho: 
 
1. A la vida, a su identidad, a su integridad moral, psíquica y  física y a su libre desarrollo 
y bienestar. El concebido es sujeto de derecho en todo cuanto le favorece. 
 
2. A la igualdad ante la ley. Nadie debe ser discriminado por motivo de origen, raza, sexo, 
idioma, religión, opinión, 

In [64]:
pdf_text

(#67) ['La Constitución \nPREAMBULO \n \n  El  Congreso  Constituyente  Democrático,  invocando  a  Dios  Todopoderoso, \nobedeciendo  el  mandato  del  pueblo  peruano  y  recordando  el  sacrificio  de  todas  las \ngeneraciones  que  nos  han  precedido  en  nuestra  Patria,  ha  resuelto  dar  la  siguiente \nConstitución: \n \nCONSTITUCION POLITICA DEL PERU DE 1993 1 \n \n \nTITULO I \n \nDE LA PERSONA Y DE LA SOCIEDAD \n \n \nCAPITULO I \n \nDERECHOS FUNDAMENTALES DE LA PERSONA \n \nArtículo  1.- Defensa de la persona humana \nLa defensa de la persona humana y el respeto de su dignidad son el fin supremo de la \nsociedad y del Estado. \n \nArtículo  2.- Derechos fundamentales de la persona \nToda persona tiene derecho: \n \n1. A la vida, a su identidad, a su integridad moral, psíquica y  física y a su libre desarrollo \ny bienestar. El concebido es sujeto de derecho en todo cuanto le favorece. \n \n2. A la igualdad ante la ley. Nadie debe ser discriminado por motivo de origen, ra

In [71]:
pdf_text.map(lambda x: x.startswith('La Constitución \n'))

(#67) [True,True,True,True,True,True,True,True,True,True...]

In [63]:
print(pdf_text[3])

La Constitución 
20.  A  formular  peticiones,  individual  o  colectivamente,  por escrito  ante  la  autoridad 
competente, la que está obligada a dar al interesado una respuesta también por escrito 
dentro del plazo legal, bajo responsabilidad. 
 
Los miembros de las  Fuerzas Armadas y de la Policía Nacional  sólo pueden ejercer 
individualmente el derecho de petición. 
 
21. A su nacionalidad. Nadie puede ser despojado de ella.  Tampoco puede ser privado del 
derecho  de  obtener  o  de  renovar  su    pasaporte  dentro  o  fuera  del  territorio  de  la 
República. 
 
22. A la paz, a la tranquilidad, al disfrute del tiempo libre y al descanso, así como a gozar 
de un ambiente equilibrado y adecuado al desarrollo de su vida. 
 
23. A la legítima defensa. 
 
24. A la libertad y a la seguridad personales. En consecuencia: 
 
a. Nadie está obligado a hacer lo que la ley no manda, ni impedido de hacer lo que ella no 
prohíbe. 
 
b. No se permite forma alguna de restricción de la libert

In [57]:
print(t)

La Constitución 
PREAMBULO 
 
  El  Congreso  Constituyente  Democrático,  invocando  a  Dios  Todopoderoso, 
obedeciendo  el  mandato  del  pueblo  peruano  y  recordando  el  sacrificio  de  todas  las 
generaciones  que  nos  han  precedido  en  nuestra  Patria,  ha  resuelto  dar  la  siguiente 
Constitución: 
 
CONSTITUCION POLITICA DEL PERU DE 1993 1 
 
 
TITULO I 
 
DE LA PERSONA Y DE LA SOCIEDAD 
 
 
CAPITULO I 
 
DERECHOS FUNDAMENTALES DE LA PERSONA 
 
Artículo  1.- Defensa de la persona humana 
La defensa de la persona humana y el respeto de su dignidad son el fin supremo de la 
sociedad y del Estado. 
 
Artículo  2.- Derechos fundamentales de la persona 
Toda persona tiene derecho: 
 
1. A la vida, a su identidad, a su integridad moral, psíquica y  física y a su libre desarrollo 
y bienestar. El concebido es sujeto de derecho en todo cuanto le favorece. 
 
2. A la igualdad ante la ley. Nadie debe ser discriminado por motivo de origen, raza, sexo, 
idioma, religión, opinión, 